## 平均脸
![这里写图片描述](http://www.learnopencv.com/wp-content/uploads/2016/05/average_entrepreneur.jpg)
![这里写图片描述](https://www.learnopencv.com/wp-content/uploads/2016/05/average_best_actress-300x300.jpg)


py代码以及相关数据地址：https://www.learnopencv.com/wp-content/uploads/2016/05/FaceAverage.zip

最初博文地址：https://www.learnopencv.com/average-face-opencv-c-python-tutorial/
中文翻译：http://blog.csdn.net/GraceDD/article/details/51382952


中文改编地址：[《手把手：用OpenCV亲手给小扎、Musk等科技大佬们做一张“平均脸”（附Python代码）》](https://mp.weixin.qq.com/s?__biz=MjM5MTQzNzU2NA==&mid=2651654758&idx=1&sn=b60e2da0b4e9cffed660f44bd624eb9e&chksm=bd4c2df58a3ba4e32f938df33cdc780bd7041087c6f3b82cf059c036a50c12e97067a8d12815&mpshare=1&scene=1&srcid=1123eFDjNTtDFdq4GS8M2e8d#rd)

## 平均脸
在完成各个library的安装后。
第一步：将要平均的照片放入faces文档，确保图片为jpg格式。
第二步：在终端运行 python face_landmark_detection.py shape_predictor_68_face_landmarks.dat faces，并在程序运行结束后将所有faces文档中的文件复制到presidents文档中（如无法完成dlib安装，可略过该步骤，直接用文摘菌提供的素材）
第三步：在终端运行 python faceAverage.py 
这样就能看到制作成功的平均脸了！


In [ ]:
import os
import cv2
import numpy as np
import math
import sys

# Read points from text files in directory
def readPoints(path) :
    # Create an array of array of points.
    pointsArray = [];

    #List all files in the directory and read points from text files one by one
    for filePath in os.listdir(path):
        
        if filePath.endswith(".txt"):
            
            #Create an array of points.
            points = [];            
            
            # Read points from filePath
            with open(os.path.join(path, filePath)) as file :
                for line in file :
                    x, y = line.split()
                    points.append((int(x), int(y)))
            
            # Store array of points
            pointsArray.append(points)
            
    return pointsArray;

# Read all jpg images in folder.
def readImages(path) :
    
    #Create array of array of images.
    imagesArray = [];
    
    #List all files in the directory and read points from text files one by one
    for filePath in os.listdir(path):
        if filePath.endswith(".jpg"):
            # Read image found.
            img = cv2.imread(os.path.join(path,filePath));

            # Convert to floating point
            img = np.float32(img)/255.0;

            # Add to array of images
            imagesArray.append(img);
            
    return imagesArray;
                
# Compute similarity transform given two sets of two points.
# OpenCV requires 3 pairs of corresponding points.
# We are faking the third one.

def similarityTransform(inPoints, outPoints) :
    s60 = math.sin(60*math.pi/180);
    c60 = math.cos(60*math.pi/180);  
  
    inPts = np.copy(inPoints).tolist();
    outPts = np.copy(outPoints).tolist();
    
    xin = c60*(inPts[0][0] - inPts[1][0]) - s60*(inPts[0][1] - inPts[1][1]) + inPts[1][0];
    yin = s60*(inPts[0][0] - inPts[1][0]) + c60*(inPts[0][1] - inPts[1][1]) + inPts[1][1];
    
    inPts.append([np.int(xin), np.int(yin)]);
    
    xout = c60*(outPts[0][0] - outPts[1][0]) - s60*(outPts[0][1] - outPts[1][1]) + outPts[1][0];
    yout = s60*(outPts[0][0] - outPts[1][0]) + c60*(outPts[0][1] - outPts[1][1]) + outPts[1][1];
    
    outPts.append([np.int(xout), np.int(yout)]);
    
    tform = cv2.estimateRigidTransform(np.array([inPts]), np.array([outPts]), False);  # 多个二维点对之间的仿射变换矩阵（使用误差最小准则）
    
    return tform;


# Check if a point is inside a rectangle
def rectContains(rect, point) :
    if point[0] < rect[0] :
        return False
    elif point[1] < rect[1] :
        return False
    elif point[0] > rect[2] :
        return False
    elif point[1] > rect[3] :
        return False
    return True

# Calculate delanauy triangle
def calculateDelaunayTriangles(rect, points):
    # Create subdiv
    subdiv = cv2.Subdiv2D(rect);
   
    # Insert points into subdiv
    for p in points:
        subdiv.insert((p[0], p[1]));

   
    # List of triangles. Each triangle is a list of 3 points ( 6 numbers )
    triangleList = subdiv.getTriangleList();

    # Find the indices of triangles in the points array

    delaunayTri = []
    
    for t in triangleList:
        pt = []
        pt.append((t[0], t[1]))
        pt.append((t[2], t[3]))
        pt.append((t[4], t[5]))
        
        pt1 = (t[0], t[1])
        pt2 = (t[2], t[3])
        pt3 = (t[4], t[5])        
        
        if rectContains(rect, pt1) and rectContains(rect, pt2) and rectContains(rect, pt3):
            ind = []
            for j in range(0, 3):
                for k in range(0, len(points)):                    
                    if(abs(pt[j][0] - points[k][0]) < 1.0 and abs(pt[j][1] - points[k][1]) < 1.0):
                        ind.append(k)                            
            if len(ind) == 3:                                                
                delaunayTri.append((ind[0], ind[1], ind[2]))

    return delaunayTri


def constrainPoint(p, w, h) :
    p =  ( min( max( p[0], 0 ) , w - 1 ) , min( max( p[1], 0 ) , h - 1 ) )
    return p;

# Apply affine transform calculated using srcTri and dstTri to src and
# output an image of size.
def applyAffineTransform(src, srcTri, dstTri, size) :
    
    # Given a pair of triangles, find the affine transform.
    warpMat = cv2.getAffineTransform( np.float32(srcTri), np.float32(dstTri) )
    
    # Apply the Affine Transform just found to the src image
    dst = cv2.warpAffine( src, warpMat, (size[0], size[1]), None, flags=cv2.INTER_LINEAR, borderMode=cv2.BORDER_REFLECT_101 )

    return dst


# Warps and alpha blends triangular regions from img1 and img2 to img
def warpTriangle(img1, img2, t1, t2) :

    # Find bounding rectangle for each triangle
    r1 = cv2.boundingRect(np.float32([t1]))
    r2 = cv2.boundingRect(np.float32([t2]))

    # Offset points by left top corner of the respective rectangles
    t1Rect = [] 
    t2Rect = []
    t2RectInt = []

    for i in range(0, 3):
        t1Rect.append(((t1[i][0] - r1[0]),(t1[i][1] - r1[1])))
        t2Rect.append(((t2[i][0] - r2[0]),(t2[i][1] - r2[1])))
        t2RectInt.append(((t2[i][0] - r2[0]),(t2[i][1] - r2[1])))


    # Get mask by filling triangle
    mask = np.zeros((r2[3], r2[2], 3), dtype = np.float32)
    cv2.fillConvexPoly(mask, np.int32(t2RectInt), (1.0, 1.0, 1.0), 16, 0);

    # Apply warpImage to small rectangular patches
    img1Rect = img1[r1[1]:r1[1] + r1[3], r1[0]:r1[0] + r1[2]]
    
    size = (r2[2], r2[3])

    img2Rect = applyAffineTransform(img1Rect, t1Rect, t2Rect, size)
    
    img2Rect = img2Rect * mask

    # Copy triangular region of the rectangular patch to the output image
    img2[r2[1]:r2[1]+r2[3], r2[0]:r2[0]+r2[2]] = img2[r2[1]:r2[1]+r2[3], r2[0]:r2[0]+r2[2]] * ( (1.0, 1.0, 1.0) - mask )
     
    img2[r2[1]:r2[1]+r2[3], r2[0]:r2[0]+r2[2]] = img2[r2[1]:r2[1]+r2[3], r2[0]:r2[0]+r2[2]] + img2Rect




In [ ]:
'''
1.读入图 + 读入关键点信息  readPoints  readImages 
2.平均脸的眼角位置（这样其他脸，按照眼睛位置对齐）  eyecornerDst
3.新的8个初始边界点 boundaryPts （为了后续做脸谱网络用的）
4.设置初始平均脸 pointsAvg （随便找个脸68个关键点 + 8个初始点）
5.根据眼睛位置，进行人脸初步对齐
6.计算初始平均脸的脸谱网络76点（calculateDelaunayTriangles）
7.根据脸谱网络二次人脸对齐

'''
# 1.读入图 + 读入关键点信息
path = 'presidents/'

# Dimensions of output image
w = 600;
h = 600;

# Read points for all images
allPoints = readPoints(path);
allPoints

# Read all images
images = readImages(path);
images

# 2.平均脸的眼角位置（这样其他脸，按照眼睛位置对齐）
# 对齐的眼角预先设定好
# 确保两只眼睛的点都在一个水平线上，面部中心大约在离顶端三分之一高度的位置。所以我将眼角位置设为（0.3*宽，高/3）和（0.7*宽，高/3）。
eyecornerDst = [ (np.int(0.3 * w ), np.int(h / 3)), (np.int(0.7 * w ), np.int(h / 3)) ];

imagesNorm = [];
pointsNorm = [];

# 3.新的8个初始边界点 boundaryPts
# Add boundary points for delaunay triangulation  根据图像长宽、设定8个初始边界点
boundaryPts = np.array([(0,0), (w/2,0), (w-1,0), (w-1,h/2), ( w-1, h-1 ), ( w/2, h-1 ), (0, h-1), (0,h/2) ]);  

# 4. 初始平均脸
# Initialize location of average points to 0s
pointsAvg = np.array([(0,0)]* ( len(allPoints[0]) + len(boundaryPts) ), np.float32());  #　初始化平均脸

n = len(allPoints[0]);

numImages = len(images)


# 5.根据眼睛位置，进行人脸初步对齐
# Warp images and trasnform landmarks to output coordinate system,
# and find average of transformed landmarks.

for i in range(0, numImages):

    points1 = allPoints[i];

    # Corners of the eye in input image
    eyecornerSrc  = [ allPoints[i][36], allPoints[i][45] ] ;

    # Compute similarity transform  眼角对齐转换矩阵
    tform = similarityTransform(eyecornerSrc, eyecornerDst);　　# 2 * 3 转换矩阵

    # Apply similarity transformation　通过眼角对齐转换矩阵 进行人脸对齐
    img = cv2.warpAffine(images[i], tform, (w,h));

    # Apply similarity transform on points
    points2 = np.reshape(np.array(points1), (68,1,2));      # (68, 1, 2)  

    points = cv2.transform(points2, tform);   # (68, 1, 2)  * (2,3)  --> (68, 1, 2)

    points = np.float32(np.reshape(points, (68, 2)));  # (68, 2)

    # Append boundary points. Will be used in Delaunay Triangulation
    points = np.append(points, boundaryPts, axis=0)   # point（68个） + boundaryPts (8个边界点)  (76, 2)

    # Calculate location of average landmark points.
    pointsAvg = pointsAvg + points / numImages;  # (76, 2)  初始平均脸

    pointsNorm.append(points);  # 6*72*2
    imagesNorm.append(img);  


# 6.计算初始平均脸的脸谱网络76点（calculateDelaunayTriangles）
# Delaunay triangulation
rect = (0, 0, w, h);
dt = calculateDelaunayTriangles(rect, np.array(pointsAvg));  # delanauy三角计算 142*3

# 7 根据脸谱网络二次人脸对齐
# Output image
output = np.zeros((h,w,3), np.float32());

# Warp input images to average image landmarks
for i in range(0, len(imagesNorm)) :
    img = np.zeros((h,w,3), np.float32());
    # Transform triangles one by one
    for j in range(0, len(dt)) :
        tin = []; 
        tout = [];

        for k in range(0, 3) :                
            pIn = pointsNorm[i][dt[j][k]];     #需要对齐的脸的脸谱网络计算
            pIn = constrainPoint(pIn, w, h);  #　约束wh，如果有越界的，要归到图像内

            pOut = pointsAvg[dt[j][k]];      # 平均脸的脸谱网络计算
            pOut = constrainPoint(pOut, w, h);

            tin.append(pIn);  
            tout.append(pOut);

        warpTriangle(imagesNorm[i], img, tin, tout);  # 把需要对齐的脸谱网络，向平均脸脸谱网络 转换


    # Add image intensities for averaging
    output = output + img;

# Divide by numImages to get average
output = output / numImages;

# Save result
cv2.imwrite('myaverageface.png', (output * 255).astype('uint8'))
